# 딥페이크 실습

## 패키지 설치 및 코드 불러오기


In [1]:
!pip install scikit-video
!git clone https://github.com/drminix/first-order-model
%cd /content/first-order-model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.3 MB 5.2 MB/s 
Cloning into 'first-order-model'...
remote: Enumerating objects: 286, done.
remote: Total 286 (delta 0), reused 0 (delta 0), pack-reused 286
Receiving objects: 100% (286/286), 117.26 MiB | 33.28 MiB/s, done.
Resolving deltas: 100% (133/133), done.
Checking out files: 100% (63/63), done.
/content/first-order-model


## 딥러닝 모델 가져오기

구글드라이브 연결

https://drive.google.com/drive/folders/1ydlLRktwcmnXVncgE0w2FFWFnBmA1ch6?usp=sharing


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
target_folder = "/content/gdrive/MyDrive/deepfake"

Mounted at /content/gdrive


## 입력 영상과 사진

In [3]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from IPython.display import HTML
import warnings
import os
import skvideo.io
warnings.filterwarnings("ignore")

#입력사진
# source_image = imageio.imread('/content/gdrive/MyDrive/deepfake/jyw_young.jpg')
source_image = imageio.imread('/content/gdrive/MyDrive/deepfake/ogame.jpg')

#입력영상
driving_video = skvideo.io.vread('/content/gdrive/MyDrive/deepfake/jyw.mp4')

#Resize image and video to 256x256
source_image = resize(source_image, (256, 256))[..., :3]

driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


def display(source, driving, generated=None):
     fig = plt.figure(figsize=(8 + 4 * (generated is not None), 6))

     ims = []
     for i in range(len(driving)):
         cols = [source]
         cols.append(driving[i])
         if generated is not None:
             cols.append(generated[i])
         im = plt.imshow(np.concatenate(cols, axis=1), animated=True)
         plt.axis('off')
         ims.append([im])

     ani = animation.ArtistAnimation(fig, ims, interval=50, repeat_delay=1000)
     plt.close()
     return ani
    
HTML(display(source_image, driving_video).to_html5_video())

## 영상 만들기

In [4]:
from demo import load_checkpoints
import os
generator, kp_detector = load_checkpoints(config_path='config/vox-256.yaml', 
                            checkpoint_path=os.path.join(target_folder,'vox-cpk.pth.tar'))

In [7]:
!pip install imageio_ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.4 MB/s 


In [8]:
from demo import make_animation
from skimage import img_as_ubyte

predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=True)

#save resulting video
imageio.mimsave('../generated.mp4', [img_as_ubyte(frame) for frame in predictions])
#video can be downloaded from /content folder

HTML(display(source_image, driving_video, predictions).to_html5_video())

100%|██████████| 759/759 [00:31<00:00, 24.27it/s]


## 딥페이크 영상을 만들기

In [9]:
!mkdir face_images
!mkdir input_images

In [10]:
!python face_extractor.py

64053248/64040097 [==============================] - 3s 0us/step
